# Machine Learning Engineer Nanodegree
## Project: Capstone Proposal and Capstone Project
## Definition

For the final project I havee decided to join to one of competitions on Kaggle.com. 
	
It was launched by [Home Credit Group](http://www.homecredit.net), an international consumer finance provider. 
It focuses on responsible lending primarily to people with little or no credit history. 
Many people struggle to get loans due to insufficient or non-existent credit histories. 
And, unfortunately, this population is often taken advantage of by untrustworthy lenders. 
Home Credit strives to broaden financial inclusion for the unbanked population by providing 
a positive and safe borrowing experience. In order to make sure this underserved population has a positive 
loan experience, Home Credit makes use of a variety of alternative data, including telco and transactional 
information to predict their clients' repayment abilities.

### Data Description

   There are the following files in a dataset
   
•  **application_{train|test}.csv**

       o This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
	
       o Static data for all applications. One row represents one loan in our data sample.
	
•  **bureau.csv**
	
       o All client's previous credits provided by other financial institutions that were reported to Credit Bureau 
	 (for clients who have a loan in our sample).
	 
       o For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau
     	before the application date.
	
•  **bureau_balance.csv**

       o Monthly balances of previous credits in Credit Bureau.
   
       o This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table              has (#loans in sample * # of relative previous credits * # of months where we have some history observable 
	     for the previous credits) rows.
	
•  **POS_CASH_balance.csv**

       o Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
	
       o This table has one row for each month of history of every previous credit in Home Credit 
       (consumer credit and cash loans) related to loans in our sample – i.e. the table has 
       (#loans in sample * # of relative previous credits * # of months in which we have some history observable 
       for the previous credits) rows.

•  **credit_card_balance.csv**

    o Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
	
    o This table has one row for each month of history of every previous credit in Home Credit 
      (consumer credit and cash loans) related to loans in our sample – i.e. the table has 
      (#loans in sample *# of relative previous credit cards *# of months where we have some history observable 
      for the previous credit card) rows.
    
•  **previous_application.csv**

    o All previous applications for Home Credit loans of clients who have loans in our sample.
   
    o There is one row for each previous application related to loans in our data sample.
   
•  **installments_payments.csv**

    o Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample. 
   
    o There is a) one row for every payment that was made plus b) one row each for missed payment.
   
    o One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous 
	Home Credit credit related to loans in our sample.
	
•  **HomeCredit_columns_description.csv**

    o This file contains descriptions for the columns in the various data files.

All files can be downloaded from <https://www.kaggle.com/c/home-credit-default-risk/data>

In [17]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from sklearn.cross_validation import ShuffleSplit

# Import supplementary visualizations code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Load the Credit Bank  dataset
train=pd.read_csv("./data/application_train.csv")
test=pd.read_csv("./data/application_test.csv")
 
# Success
print "Training dataset has {} data points with {} variables each.".format(*train.shape)
print "Testing dataset has {} data points with {} variables each.".format(*test.shape)

train.head()

Training dataset has 307511 data points with 122 variables each.
Testing dataset has 48744 data points with 121 variables each.


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
